In [15]:
import torch
import torchvision
import numpy as np
import module
import vgg

In [20]:
model = torchvision.models.vgg16(pretrained=True)
# model = vgg.vgg16(pretrained=True)
model = torch.nn.DataParallel(model).cuda()

In [21]:
quantization_factor = [None for i in range(16)]
pruning_factor = [3 for i in range(16)]

In [23]:
# def weight_processing(model, quantization_factor, pruning_factor):
state_dict = model.state_dict()

counter = 0
total = 0
sum = 0
for module_name, layer in model.named_modules():
    if type(layer) == torch.nn.modules.conv.Conv2d \
    or type(layer) == torch.nn.modules.linear.Linear or type(layer) == torch.nn.modules.batchnorm.BatchNorm2d or type(layer) == module.QuantizeConv2d or type(layer) == module.QuantizeLinear: 
        for layer_name, parameter in layer.named_parameters():
            if quantization_factor[counter] != None:
                state_dict[module_name + '.' + layer_name] = module.quantize(parameter, quantization_factor[counter])
            if pruning_factor[counter] != None:
                state_dict[module_name + '.' + layer_name] = state_dict[module_name + '.' + layer_name] * module.pruning(parameter, pruning_factor[counter])
                sum += len(torch.nonzero(state_dict[module_name + '.' + layer_name]))
                total += len(state_dict[module_name + '.' + layer_name].view(-1))
                print('['+str(counter)+'] layer pruning rate : ' + str((1-(len(torch.nonzero(state_dict[module_name + '.' + layer_name])) / 
                                                                           len(state_dict[module_name + '.' + layer_name].view(-1))))*100))
        counter += 1

# print('total layer pruning rate : ' + str((1-sum/total)*100))

model.load_state_dict(state_dict)


[0] layer pruning rate : 47.16435185185185
[0] layer pruning rate : 14.0625
[1] layer pruning rate : 96.240234375
[1] layer pruning rate : 51.5625
[2] layer pruning rate : 97.45008680555556
[2] layer pruning rate : 64.84375
[3] layer pruning rate : 99.20450846354166
[3] layer pruning rate : 67.1875
[4] layer pruning rate : 99.69991048177084
[4] layer pruning rate : 83.203125
[5] layer pruning rate : 99.94998508029514
[5] layer pruning rate : 82.421875
[6] layer pruning rate : 99.90251329210069
[6] layer pruning rate : 81.640625
[7] layer pruning rate : 99.94413587782118
[7] layer pruning rate : 83.59375
[8] layer pruning rate : 99.9945322672526
[8] layer pruning rate : 73.4375
[9] layer pruning rate : 99.99508327907986
[9] layer pruning rate : 72.8515625
[10] layer pruning rate : 99.99656677246094
[10] layer pruning rate : 74.609375
[11] layer pruning rate : 99.99949137369791
[11] layer pruning rate : 58.984375
[12] layer pruning rate : 99.99983045789931
[12] layer pruning rate : 66.60

In [5]:
# print(model.state_dict()['module.features.0.weight'][0])
weight_processing(model, quantization_weight, pruning_weight)
# print(model.state_dict()['module.features.0.weight'][0])

NameError: name 'weight_processing' is not defined

In [16]:
torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [30]:
len(temp.view(-1))

27